# Kacper Kafara

## Lab 4

# Uwaga

Hash ostatniego commita w repo z zadaniami gdy pobrałem tekst zadań: `e3d6681`

Link do ostatniego commita: [link](https://github.com/apohllo/nlp/commit/e3d6681744adf3dd9025ce5386ab622a30c7e42f)

## Zadanie 1

*Use SpaCy tokenizer API to tokenize the text from the FIQA corpus (from the 1 lab).*

In [1]:
import numpy as np
import spacy
import textacy
import datasets as ds

dev_text_count = 10

/home/kkafara/.cache/virtualenvs/python-3.11-venv-ds/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_path = 'clarin-knext/fiqa-pl'
dataset_name = 'corpus'

In [3]:
fiqa_ds = ds.load_dataset(dataset_path, dataset_name)[dataset_name]
fiqa_ds

Dataset({
    features: ['_id', 'title', 'text'],
    num_rows: 57638
})

In [4]:
texts = fiqa_ds['text']
texts[0]

'Nie mówię, że nie podoba mi się też pomysł szkolenia w miejscu pracy, ale nie możesz oczekiwać, że firma to zrobi. Szkolenie pracowników to nie ich praca – oni tworzą oprogramowanie. Być może systemy edukacyjne w Stanach Zjednoczonych (lub ich studenci) powinny trochę martwić się o zdobycie umiejętności rynkowych w zamian za ich ogromne inwestycje w edukację, zamiast wychodzić z tysiącami zadłużonych studentów i narzekać, że nie są do niczego wykwalifikowani.'

Jako, że nie widziałem możliwości, aby zmodyfikować tekst w Tokenie (a potrzebujemy zrobić lowercase),
więc konwertuję najpierw cały tekst, a potem tokenizuję.

In [5]:
for i, text in enumerate(texts):
    texts[i] = text.lower()

texts[0]

'nie mówię, że nie podoba mi się też pomysł szkolenia w miejscu pracy, ale nie możesz oczekiwać, że firma to zrobi. szkolenie pracowników to nie ich praca – oni tworzą oprogramowanie. być może systemy edukacyjne w stanach zjednoczonych (lub ich studenci) powinny trochę martwić się o zdobycie umiejętności rynkowych w zamian za ich ogromne inwestycje w edukację, zamiast wychodzić z tysiącami zadłużonych studentów i narzekać, że nie są do niczego wykwalifikowani.'

In [6]:
# Tokenizer dla j. polskiego
spacy_nlp = spacy.load('pl_core_news_sm')
tokenizer = spacy_nlp.tokenizer

# Tokenizacja
tokenized_texts = list(tokenizer.pipe(texts, batch_size=200))

# Zadanie 2

*Compute bigram counts of downcased tokens*

Korzystam z `textacy`. Nie znam tej biblioteki, trzymam kciuki, że działa dobrze.

Potrzebuję stworzyć `SpanWrapper`, bo tokenizer + `textacy` zwracają `Span`, który jest haszowany z wykorzystaniem doca, i pozycji tokenów w tym docu ==> nawet jeżeli tokeny mają taki sam tekst,
to jeśli są w różnych miejsach w tekście (czyli 100% przypadków) to mają różne hashe ==> zliczanie słownikiem nie działa

Referencja w kodzie spaCy: https://github.com/adrianeboyd/spaCy/blob/120601ddbe675a6acc5cfa2900b48a1fe4b23399/spacy/tokens/span.pyx#L149-L150


In [7]:
from collections import Counter, defaultdict

class SpanWrapper:
    def __init__(self, bigram: spacy.tokens.span.Span):
        self.bigram = bigram
        self.hash_value = hash(str(bigram))

    def __eq__(self, other: object) -> bool:
        return self.hash_value == other.hash_value

    def __hash__(self) -> int:
        return self.hash_value

    def __str__(self) -> str:
        return str(self.bigram)

def wrap_spans(span: spacy.tokens.span.Span) -> SpanWrapper:
    return SpanWrapper(span)

# Łączę wszystkie teksty w jeden duży, bo dodawanie słowników jest niesamowicie powolne...
joined_doc = spacy.tokens.doc.Doc.from_docs(tokenized_texts)
del tokenized_texts


bigram_counter = Counter(map(wrap_spans, textacy.extract.basics.ngrams(joined_doc, n=2, min_freq=1, filter_stops=False, filter_punct=False, filter_nums=False)))

In [8]:
def most_common_bigrams_from_counter(counter: Counter, n: int = 10) -> list[tuple[SpanWrapper, int]]:
    return list(map(lambda swrap: (swrap[0].bigram, swrap[1]), counter.most_common(n)))

most_common_bigrams = most_common_bigrams_from_counter(bigram_counter, 10)
print(most_common_bigrams)
print(len(bigram_counter))

[(, że, 86093), (, a, 32643), (, ale, 32549), (, aby, 28290), (, które, 22362), (. jeśli, 20292), (, co, 18674), (to,, 16612), (, jeśli, 15689), (jest to, 15642)]
2039426


# Zadanie 3

*Discard bigrams containing characters other than letters. Make sure that you discard the invalid entries after computing the bigram counts.*

In [9]:
def is_alpha_or_whitespace(string: str) -> bool:
    return all(map(lambda x: x.isalpha() or x.isspace(), string))

letter_only_bigram_counts = Counter({
    swrap: count for swrap, count in bigram_counter.items() if is_alpha_or_whitespace(swrap.bigram.text)
})

most_common_bigrams_from_counter(letter_only_bigram_counts), len(letter_only_bigram_counts)

([(jest to, 15642),
  (nie jest, 11242),
  (po prostu, 9762),
  (nie ma, 8626),
  (w tym, 7808),
  (się z, 6060),
  (i nie, 5203),
  (w przypadku, 5137),
  (się na, 5133),
  (że nie, 5012)],
 1568709)

# Zadanie 4

*Use pointwise mutual information to compute the measure for all pairs of words.*

wg Wikipedii PMI to:

![Alt text](image.png)

Jako, że nie jestem pewien jak to powinno być liczone, to przyjmuję następująco:

Przestrzeń zdarzeń to zbiór wszystkich bigramów.

Wtedy p(x) określam jako prawdopodobieństwo wystąpienia słowa x w bigramie --> (liczba bigramów w których jest x) / (liczba wszystkich bigramów).

p(x, y) określam jako wystąpienia bigramu "x y" przez liczbę wszystkich bigramów

Dodatkowo jako "liczbę wszystkich bigramów" przyjmuję liczbę bigramów **przed odfiltrowaniem** tych bigramów, które nie zawierają tylko liter (i spacji pomiędzy). Tak sugeruje mi drugie zdanie w treści zadania 3.

In [10]:
from typing import Dict
from math import log2
from tqdm import tqdm
from collections import defaultdict
from pprint import pprint

def prob_bigram(span_wrap: SpanWrapper, bigram_counter: Counter, bigrams_count: int) -> float:
    return bigram_counter[span_wrap] / bigrams_count

    
def pmi(span_wrap: SpanWrapper, bigram_counter: Counter, bigrams_count: int, word_occurences: Dict[str, int]) -> float:
    words = span_wrap.bigram.text.split(' ')
    assert len(words) == 2, f"Expected two words in bigram, got {words}"
    return log2(prob_bigram(span_wrap, bigram_counter, bigrams_count)) - log2((word_occurences[words[0]] /  bigrams_count) * (word_occurences[words[1]] / bigrams_count))

bigrams_count = bigram_counter.total()
word_occurences = defaultdict(int)

# Tutaj zliczam tylko słowa, które będę potem odpytywał.
# Korzystam z count z przefiltrowanych bigramów, bo inaczej
# robi się to ciężkie obliczeniowo.
# Liczę, że za bardzo się to nie rozjedzie.
for span_wrap, count in letter_only_bigram_counts.items():
    words = span_wrap.bigram.text.split(' ')
    word_occurences[words[0]] += count
    word_occurences[words[1]] += count

pmi_map = {
    span_wrap: pmi(span_wrap, bigram_counter, bigrams_count, word_occurences) for span_wrap in tqdm(letter_only_bigram_counts.keys())
}

pprint(list(pmi_map.items())[:10])

100%|██████████| 1568709/1568709 [00:05<00:00, 307962.38it/s]

[(<__main__.SpanWrapper object at 0x7fc3b820b5d0>, 3.2934839651841195),
 (<__main__.SpanWrapper object at 0x7fc3a844d290>, 1.0797346306395657),
 (<__main__.SpanWrapper object at 0x7fc3a844ff10>, 3.0118232161899385),
 (<__main__.SpanWrapper object at 0x7fc3a844ff50>, 7.982189514658092),
 (<__main__.SpanWrapper object at 0x7fc3a844fe50>, 2.8144940094670368),
 (<__main__.SpanWrapper object at 0x7fc3a844fbd0>, -0.5551592742439677),
 (<__main__.SpanWrapper object at 0x7fc3a844d950>, 0.9311125432685401),
 (<__main__.SpanWrapper object at 0x7fc3a844fe10>, 4.460903890284698),
 (<__main__.SpanWrapper object at 0x7fc3a844c790>, -0.2073479719979474),
 (<__main__.SpanWrapper object at 0x7fc3a844ca10>, 1.7038223509878065)]


# Zadania 5, 6

*Sort the word pairs according to that measure in the descending order and determine top 10 entries.*

*Filter bigrams with number of occurrences lower than 5. Determine top 10 entries for the remaining dataset (>=5 occurrences).*

In [11]:
pmi_list = list(pmi_map.items())
pmi_list.sort(key=lambda entry: entry[1], reverse=True)
pprint(list(map(lambda entry: entry[0].bigram, pmi_list[:10])))

[али устройстройството,
 program킨 diegimas,
 skambinkite mums,
 przetwarzającemu przejmującemu,
 finest liquids,
 роект btchash,
 попробуй сам,
 marriner eccles,
 cort ammon,
 zjebali mroźny]



Teraz odfiltorwujemy bigramy o małej liczbie wystąpień.

In [12]:
pmi_list = [entry for entry in pmi_list if letter_only_bigram_counts[entry[0]] >= 5]
no_lt_result = [entry[0].bigram for entry in pmi_list][:10]
pprint(no_lt_result)

[моя группа,
 autot ldr,
 которые платят,
 остались вопросы,
 инарные опционы,
 олимп трейд,
 мою команду,
 nonresident external,
 lightning eliminators,
 stucco veneziano]


Tak samo. "metali szlachetnych", "due diligence", "klęski żywiołowe", "brzytwa Ockhama" -- te zwroty rzeczywiście występują często razem w j. polskim ==> wygląda do sensownie.

^ -- powyższe dostałem gdy nie puściłem liczenia na całym korpusie, a na pierwszych 10k tekstach...

Na całym korpisue wyniki zostały "zaśmiecone" wstawkami z innych języków!

# Zadanie 7

*Use SpaCy to lemmatize and tag the sentences in the corpus.*

Przepuszczam nasze teksty poprostu przez cały Pipeline dla polskiego, jako że w "środku" jest lematyzacja i tagowanie: https://spacy.io/models/pl#pl_core_news_sm


In [13]:
processed_texts = list(spacy_nlp.pipe(texts, batch_size=100))

In [14]:
for token in processed_texts[0][:2]:
    print(token, token.tag_, token.lemma_)

nie QUB nie
mówię FIN mówić


^ Wygląda sensownie

# Zadanie 8

Using the tagged corpus compute bigram statistic for the tokens containing: 
* a. lemmatized, downcased word 
* b. morphosyntactic category of the word (subst, fin, adj, etc.)

Tworzymy bigramy

In [15]:
from collections import Counter, defaultdict

class SpanWrapperLT:
    def __init__(self, bigram: spacy.tokens.span.Span):
        self.bigram = bigram
        self.hash_string = " ".join([f'{token.lemma_}:{token.tag_}' for token in self.bigram])
        self.hash_value = hash(self.hash_string)

    def tokens(self):
        return (self.bigram.doc[self.bigram.start], self.bigram.doc[self.bigram.end-1])

    def __eq__(self, other: object) -> bool:
        return self.hash_value == other.hash_value

    def __hash__(self) -> int:
        return self.hash_value

    def __str__(self) -> str:
        return self.hash_string
    
    def __repr__(self) -> str:
        return self.__str__()

def wrap_spans_lt(span: spacy.tokens.span.Span) -> SpanWrapperLT:
    return SpanWrapperLT(span)

# Łączę wszystkie teksty w jeden duży, bo dodawanie słowników jest niesamowicie powolne...
joined_doc = spacy.tokens.doc.Doc.from_docs(processed_texts)
# del processed_texts


bigram_lt_counter = Counter(map(wrap_spans_lt, textacy.extract.basics.ngrams(joined_doc, n=2, min_freq=1, filter_stops=False, filter_punct=False, filter_nums=False)))

In [16]:
most_common_bigrams_lt = most_common_bigrams_from_counter(bigram_lt_counter, 10)
pprint(most_common_bigrams_lt)
print(len(bigram_lt_counter))

[(, że, 75938),
 (, które, 53518),
 (, a, 32492),
 (, ale, 31389),
 (to,, 26532),
 (, aby, 26512),
 (. jeśli, 18613),
 (nie są, 18313),
 (, jeśli, 15328),
 (, ponieważ, 14998)]
1801315


Odfiltrowuję bigramy zawierające znaki inne niż litery (+ spacja) i liczę PMI

In [17]:
letter_only_bigram_lt_counts = Counter({
    swrap: count for swrap, count in bigram_lt_counter.items() if is_alpha_or_whitespace(swrap.bigram.text)
})

most_common_bigrams_from_counter(letter_only_bigram_lt_counts), len(letter_only_bigram_lt_counts)

([(nie są, 18313),
  (nie ma, 13376),
  (jest to, 11298),
  (po prostu, 9579),
  (w tych, 9005),
  (mogą być, 7178),
  (się z, 6631),
  (nie możesz, 6594),
  (w którym, 6584),
  (na to, 6487)],
 1306985)

In [18]:
bigrams_count = bigram_lt_counter.total()
word_occurences = defaultdict(int)

for span_wrap, count in letter_only_bigram_lt_counts.items():
    words = span_wrap.bigram.text.split(' ')
    word_occurences[words[0]] += count
    word_occurences[words[1]] += count

pmi_lt_map = {
    span_wrap: pmi(span_wrap, bigram_lt_counter, bigrams_count, word_occurences) for span_wrap in tqdm(letter_only_bigram_lt_counts.keys())
}

100%|██████████| 1306985/1306985 [00:04<00:00, 319511.56it/s]


Sortuję i wypisuję bigramy o największym PMI

In [19]:
pmi_lt_list = list(pmi_lt_map.items())
pmi_lt_list.sort(key=lambda entry: entry[1], reverse=True)

Odfiltrowuję bigramy o niskiej liczbie wystąpień

In [20]:
pmi_lt_list = [entry for entry in pmi_lt_list if letter_only_bigram_lt_counts[entry[0]] >= 5]
lt_result = [entry[0].bigram for entry in pmi_lt_list][:10]
pprint(lt_result)

[моя группа,
 autot ldr,
 rurociągami naftowymi,
 biegłymi rewidentami,
 остались вопросы,
 которые платят,
 инарные опционы,
 олимп трейд,
 мою команду,
 skuterze śnieżnym]


# Zadania 11, 12

*Group the bigrams by morphosyntactic tag, i.e. a pair of words belongs to a given group if all pairs have the same syntactic category for the first and the second word.
E.g. one group would be words with subst as the first words and adj as the second word.*

*Print top-10 categories (sort them by total count of bigrams) and print top-5 pairs for each category.*

In [21]:
class GroupKey:
    def __init__(self, wrapper: SpanWrapperLT):
        self.wrapper = wrapper
        token_1, token_2 = wrapper.tokens()
        self.hash_string = token_1.tag_ + ':' + token_2.tag_
        self.hash_value = hash(self.hash_string)

    def __eq__(self, other: object) -> bool:
        return self.hash_string == other.hash_string

    def __hash__(self):
        return self.hash_value
    
    def __repr__(self) -> str:
        return self.hash_string
        
keyed_wrappers = [GroupKey(wrap) for wrap in letter_only_bigram_lt_counts.keys()]

grouped_by_tags = defaultdict(list)
for keyed_wrapper in keyed_wrappers:
    # print(keyed_wrapper)
    grouped_by_tags[keyed_wrapper].append(keyed_wrapper.wrapper) 

Top 10 kategorii:

In [22]:
items_by_tag = list(grouped_by_tags.items())
items_by_tag.sort(key=lambda kv: len(kv[1]), reverse=True)
for cat, items in items_by_tag[:10]:
    print(cat, [item.bigram for item in items[:5]])

SUBST:SUBST [miejscu pracy, niczego wykwalifikowani, strony rynku, ramach dfa, dfa banki]
ADJ:SUBST [ogromne inwestycje, fałszywym ocenom, dodatkową kontrolą, nowsze kontrole, należytej staranności]
SUBST:ADJ [systemy edukacyjne, stanach zjednoczonych, umiejętności rynkowych, ratingach kredytowych, instrumentu finansowego]
SUBST:FIN [intencją jest, cdo jest, fsa mogą, co chcą, składek jest]
PREP:SUBST [w miejscu, w stanach, w edukację, z tysiącami, do niczego]
FIN:SUBST [tworzą oprogramowanie, może systemy, uniemożliwiają instytucjom, jest plus, jest to]
SUBST:PREP [inwestycje w, ocenom poza, kontrolą ze, staranności przy, fsa dla]
INF:SUBST [ograniczyć to, użyć fsa, użyć planu, kupić kontrakt, dokonać dostawy]
CONJ:SUBST [i narzekać, więc nic, to samsung, lub firma, lub spółka]
GER:SUBST [szkolenie pracowników, zdobycie umiejętności, opłacania składek, wyłączeniem wartości, zachowanie instrumentów]


# Zadanie 13

*Create a table comparing the results for copora without and with tagging and lemmatization.*

In [23]:
import polars as pl

pl.DataFrame({
    'w/o lemmatization and taggging': no_lt_result,
    'w/ lemmatization and tagging': lt_result
}).with_row_count()

row_nr,w/o lemmatization and taggging,w/ lemmatization and tagging
u32,object,object
0,моя группа,моя группа
1,autot ldr,autot ldr
2,которые платят,rurociągami naftowymi
3,остались вопросы,biegłymi rewidentami
4,инарные опционы,остались вопросы
5,олимп трейд,которые платят
6,мою команду,инарные опционы
7,nonresident external,олимп трейд
8,lightning eliminators,мою команду


In [24]:
pl.DataFrame({
    str(col): [item.bigram for item in items[:5]] for col, items in items_by_tag[:10]
}).with_row_count()

row_nr,SUBST:SUBST,ADJ:SUBST,SUBST:ADJ,SUBST:FIN,PREP:SUBST,FIN:SUBST,SUBST:PREP,INF:SUBST,CONJ:SUBST,GER:SUBST
u32,object,object,object,object,object,object,object,object,object,object
0,miejscu pracy,ogromne inwestycje,systemy edukacyjne,intencją jest,w miejscu,tworzą oprogramowanie,inwestycje w,ograniczyć to,i narzekać,szkolenie pracowników
1,niczego wykwalifikowani,fałszywym ocenom,stanach zjednoczonych,cdo jest,w stanach,może systemy,ocenom poza,użyć fsa,więc nic,zdobycie umiejętności
2,strony rynku,dodatkową kontrolą,umiejętności rynkowych,fsa mogą,w edukację,uniemożliwiają instytucjom,kontrolą ze,użyć planu,to samsung,opłacania składek
3,ramach dfa,nowsze kontrole,ratingach kredytowych,co chcą,z tysiącami,jest plus,staranności przy,kupić kontrakt,lub firma,wyłączeniem wartości
4,dfa banki,należytej staranności,instrumentu finansowego,składek jest,do niczego,jest to,fsa dla,dokonać dostawy,lub spółka,zachowanie instrumentów


# Zadanie 14

Answer the following questions:
* Why do we have to filter the bigrams, rather than the token sequence?
* What types of expressions are discovered by the methods.
* Can you devise a different type of filtering that would yield better results?

* Filtrujemy bigramy o niskiej liczbie wystąpień, ponieważ chcemy wyeliminować sytuacje w których pojedyncze zestawienia słów uzysukją bardzo duży wskaźnik PMI w dość przypadkowy sposób. Nie za bardzo rozumiem
o jakie "filtrowanie sekwencji tokenów" chodziło autorowi pytania, więc pomijam.

* Zakładam, że "metody" to bez lematyzacji + tagownia oraz z. Moje ogólne wrażenie jest takie, ze obie te metody faktycznie wskazały te bigramy, które faktycznie często występują wspólnie w języku polskim ("naturalne kolokacje"?).
  Część pozycji w top 10 obu metod pokrywa się (na różnych miejscach, ale te same wartości): "litr mleka", "dyrektor generalny", "rezerwa federalna", "instrumenty pochodne".

* Zwiększenie limitu liczby wystąpień (10, 20, 30)?. Też zasadne jest pytanie, co znaczy "lepszy wynik" tutaj?


**UWAGA** Powyższe odpowiedzi napisałem puszczając obliczenia dla pierwszych 10k tekstów z korpusu. Wtedy wygląda to naprawdę fajnie. Gdy puściłem na całości, wyniki zostały zupełnie zniekształcone i zaśmiecone,
w szczególności wstawkami z innych języków.

Możnaby zatem próbować w "jakiś sposób" (korzystając z lematyzacji możemy rozpoznawać czy fraza jest po polsku czy nie, dosyc łatwo (są słowniki)) filtrować wyrażenia w obcym języku (jeżeli zależy nam na wynikach dla j. polskiego).
